In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
# from bo_methods_lib.bo_methods_lib.bo_functions_generic import gen_theta_set, clean_1D_arrays
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
#Get Best Data
date_time_str = "2023/09/06/"
name_cs_str = "CS1" 
save_csv = False
study_id = "SF"

#Get theta_true
file_path = date_time_str + "Data_Files/" + name_cs_str + "_BO_method_C2_sep_fact_1.0"
fileObj = open(file_path + ".pickle", 'rb')
results = pickle.load(fileObj) 
fileObj.close()
theta_true = results[0].simulator_class.theta_true

df_best = get_best_data(date_time_str, name_cs_str, study_id, theta_true, save_csv)
df_best

,BO Iter,Run Number,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time,L2 Norm Theta
65,1,2,1.550589e+00,1.356566e-07,0.000000,"[0.52631579, 0.73684211]",-2.118619e-01,[0.01388258],"[1.00726474, -1.0198189]",[0.01388258],[ 1.00726474 -1.0198189 ],0.117507,0.7,A1,2,0.235014,0.021108
379,4,12,4.386349e-01,9.600000e-01,0.381856,"[0.98522072, -1.01760162]",1.040027e+00,[0.01266045],"[0.98531305, -1.01757621]",[0.01266045],[ 0.98531305 -1.01757621],0.165920,0.5,B1,7,1.161442,0.022905
1136,2,9,3.168155e-05,1.000000e+00,0.000005,"[-0.52631579, -0.31578947]",1.082590e-05,[1.29852061e-05],"[0.99898453, -0.99971959]",[1.29852061e-05],[ 0.99898453 -0.99971959],1.060315,0.9,A2,4,4.241260,0.001053
3957,35,11,6.458391e-08,1.000000e+00,52.113367,"[0.99993208, -0.99997925]",5.939372e-08,[6.94099683e-08],"[0.99992529, -0.99998001]",[6.94099683e-08],[ 0.99992529 -0.99998001],63.639614,0.9,B2,36,2291.026099,0.000077
4711,37,8,1.390047e-07,2.151973e+09,5232.881352,"[0.99989584, -1.00001105]",1.619989e-08,[1.24068023e-07],"[0.99989144, -1.00001352]",[1.24068023e-07],[ 0.99989144 -1.00001352],3.739789,0.5,C2,38,142.112000,0.000109


In [4]:
#Get Median Data
date_time_str = "2023/09/06/"
name_cs_str = "CS1" 
save_csv = False
study_id = "SF"

#Get theta_true
file_path = date_time_str + "Data_Files/" + name_cs_str + "_BO_method_C2_sep_fact_1.0"
fileObj = open(file_path + ".pickle", 'rb')
results = pickle.load(fileObj) 
fileObj.close()
theta_true = results[0].simulator_class.theta_true

get_median_data(date_time_str, name_cs_str, study_id, theta_true, save_csv)

,BO Iter,Run Number,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time,L2 Norm Theta
31,1,0,1.550589e+00,4.510930e-04,0.000000e+00,"[-0.94736842, 2.0]",-6.325566e+00,[10.92386756],"[2.0, -0.83515894]",[10.92386756],[ 2. -0.83515894],0.085861,0.6,A1,2,0.171722,1.013495
49,1,9,1.550589e+00,3.476730e-06,0.000000e+00,"[-0.94736842, 2.0]",-6.325566e+00,[10.92386756],"[2.0, -0.83515894]",[10.92386756],[ 2. -0.83515894],0.085824,0.6,A1,2,0.171648,1.013495
562,24,9,-3.512854e+00,7.600000e-01,1.170374e-03,"[-0.10526316, 0.94736842]",1.000000e+00,[117.42245899],"[-1.46099917, 0.29316404]",[0.02978053],[-1.55023054 1.10046108],0.064834,0.6,B1,29,1.880183,2.780070
1218,6,8,3.364983e-05,1.000000e+00,1.009975e-05,"[0.73682938, -0.7368704]",6.120982e-06,[1.74205492e-05],"[1.00124778, -0.99976667]",[1.74205492e-05],[ 1.00124778 -0.99976667],0.989334,1.0,A2,7,6.925339,0.001269
1232,6,10,3.364983e-05,1.000000e+00,1.009975e-05,"[0.73682938, -0.7368704]",6.120982e-06,[1.74205492e-05],"[1.00124778, -0.99976667]",[1.74205492e-05],[ 1.00124778 -0.99976667],2.447887,1.0,A2,7,17.135210,0.001269
3515,27,14,2.460467e-07,1.000000e+00,5.202533e+01,"[0.99985047, -1.00000963]",6.154629e-08,[2.57660668e-07],"[0.99984077, -1.00001101]",[2.57660668e-07],[ 0.99984077 -1.00001101],167.591516,0.8,B2,38,6368.477618,0.000160
6099,35,12,1.550589e+00,5.591191e+253,1.516616e+255,"[-0.94736842, -1.78947368]",9.056790e-05,[2.05736054e-05],"[0.99974185, -0.99923482]",[6.51988668e-07],[ 0.99974331 -0.99923404],3.254646,0.7,C2,38,123.676541,0.000808


In [5]:
#Set Parameters
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
bo_meth_list = ["A1", "B1", "A2", "C2"]
ep_meth_list = ["CONSTANT", "DECAY", "BOYLE"]
study_id = "EP"

#Run EP Bias Experiment Analysis
analyze_ep_sep_fact_study(date_time_str, bo_meth_list, ep_meth_list, study_id, name_cs_str, save_csv)

,BO Method,Best EP Method,Min SSE,Min SSE Act,theta_1,theta_2,BO Restart,BO Iter
0,A1,BOYLE,4.276449e-02,1.861867e-01,1.135790,-0.992728,2.0,3.0
1,B1,DECAY,9.439487e-02,4.105523e-02,1.061143,-1.010391,14.0,31.0
2,A2,CONSTANT,2.360463e-07,1.503054e-06,1.000157,-0.999808,1.0,5.0
3,C2,BOYLE,8.030039e-08,9.989756e-07,0.999800,-0.999867,10.0,9.0


In [6]:
#Set Parameters
date_time_str = "2023/09/05/"
save_csv = False
name_cs_str = "CS1"      
bo_meth_list = ["A1", "B1", "A2", "B2", "C2"]
ep_meth_list = ["CONSTANT", "DECAY", "BOYLE", "JASRASARIA"]
study_id = "EP"

#Get all ep_data        
get_all_ep_sep_fact_data(date_time_str, bo_meth_list, ep_meth_list, study_id, name_cs_str, save_csv)

,Run Number,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,EP Method,BO Method,Max Evals,Total Run Time
BO Iter,,,,,,,,,,,,,,,
0,0,5.342819,1,0.0,"[1.1578947368421053, -1.5789473684210527]",0.836633,[0.08118837017924292],"[1.0650997739495396, -0.9662149682915333]",[0.08118837017924292],"[1.0650997739495396, -0.9662149682915333]",0.171955,CONSTANT,A1,2,0.343911
1,0,5.342819,1,0.0,"[0.10526315789473673, 1.3684210526315788]",1.538531,[0.27285178243012775],"[0.9598223367766308, -0.9131076839163043]",[0.08118837017924292],"[1.0650997739495396, -0.9662149682915333]",0.181924,CONSTANT,A1,2,0.363849
0,1,5.342819,1,0.0,"[-0.736842105263158, 1.3684210526315788]",0.836633,[0.08118856921646599],"[1.0650998147294313, -0.9662149047662576]",[0.08118856921646599],"[1.0650998147294313, -0.9662149047662576]",0.179331,CONSTANT,A1,2,0.358661
1,1,5.342819,1,0.0,"[-0.10526315789473695, 1.1578947368421053]",0.40207,[0.20363308352309867],"[1.1426973619099514, -1.0004762773074531]",[0.08118856921646599],"[1.0650998147294313, -0.9662149047662576]",0.174258,CONSTANT,A1,2,0.348515
0,2,5.342819,1,0.0,"[1.789473684210526, -0.10526315789473695]",0.836633,[0.08118929794577531],"[1.0651001210508368, -0.9662147611690274]",[0.08118929794577531],"[1.0651001210508368, -0.9662147611690274]",0.153579,CONSTANT,A1,2,0.307158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,12,5.342819,0.001388,0.0,"[-0.9473684210526316, 0.10526315789473673]",0.000436,[0.005806025042263389],"[1.0240555515822893, -0.9992460023603259]",[0.005806025042263389],"[1.0240555515822893, -0.9992460023603259]",1.076411,JASRASARIA,C2,2,2.152822
0,13,5.342819,0.00155,0.0,"[1.789473684210526, -0.10526315789473695]",0.000454,[0.006815831748756041],"[1.026091145176844, -0.999504336975101]",[0.006815831748756041],"[1.026091145176844, -0.999504336975101]",1.064305,JASRASARIA,C2,2,2.12861
1,13,5.342819,0.001386,0.0,"[1.5789473684210522, 0.3157894736842106]",0.000966,[0.007675471626653029],"[1.0270776725007733, -0.9968216349649657]",[0.006815831748756041],"[1.026091145176844, -0.999504336975101]",1.13037,JASRASARIA,C2,2,2.260741


In [8]:
#Separation factor experiment analysis
date_time_str = "2023/09/06/"
save_csv = False
name_cs_str = "CS1"  
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precision = 1
sep_fact_list = [f"{number:.{precision}f}" for number in f]
bo_meth_list = ["A1", "B1", "A2", "B2", "C2"]
study_id = "SF"

#Run Sep Fact Experiment Analysis
analyze_ep_sep_fact_study(date_time_str, bo_meth_list, sep_fact_list, study_id, name_cs_str, save_csv)

,BO Method,Best Sep Fact,Min SSE,Min SSE Act,theta_1,theta_2,BO Restart,BO Iter
0,A1,0.7,1.126336e-02,9.655551e-02,0.946983,-1.044868,4.0,2.0
1,B1,1.0,1.142865e+00,4.906132e-02,1.065893,-0.987118,15.0,1.0
2,A2,0.5,1.974786e-06,1.554689e-05,0.998782,-1.000144,1.0,4.0
3,B2,0.5,1.924174e-08,1.077601e-07,0.999898,-1.000012,1.0,38.0
4,C2,0.5,1.619989e-08,1.240680e-07,0.999891,-1.000014,9.0,38.0


In [10]:
#Separation factor experiment analysis
date_time_str = "2023/09/06/"
name_cs_str = "CS1" 
save_csv = False
f = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
precision = 1
sep_fact_list = [f"{number:.{precision}f}" for number in f]
bo_meth_list = ["A1", "B1", "A2", "B2", "C2"]
study_id = "SF"

#get all SF Data
get_all_ep_sep_fact_data(date_time_str, bo_meth_list, sep_fact_list, study_id, name_cs_str, save_csv)

,Run Number,Best Error,Exploration Bias,Max EI,Theta Max EI,Min Obj,Min Obj Act,Theta Min Obj,Min Obj Cum.,Theta Min Obj Cum.,Time/Iter,Sep Fact,BO Method,Max Evals,Total Run Time
BO Iter,,,,,,,,,,,,,,,
0,0,1.550589,1,0.0,"[-0.10526315789473695, 1.1578947368421053]",-8.591553,[11.0596912877479],"[2.0, -0.823457113380021]",[11.0596912877479],"[2.0, -0.823457113380021]",0.286009,0.5,A1,2,0.572019
1,0,1.550589,0.666667,0.0,"[-0.9473684210526316, 2.0]",-8.560577,[11.003694897941887],"[2.0, -0.8281848721015481]",[11.003694897941887],"[2.0, -0.8281848721015481]",0.085781,0.5,A1,2,0.171562
0,1,1.550589,0.444444,0.0,"[1.3684210526315788, 0.5263157894736841]",-8.591553,[11.059691091241215],"[2.0, -0.8234571297488728]",[11.059691091241215],"[2.0, -0.8234571297488728]",0.100281,0.5,A1,2,0.200562
1,1,1.550589,0.296296,0.0,"[0.3157894736842106, -1.1578947368421053]",-8.719846,[11.042035718854317],"[2.0, -0.8249339856566439]",[11.042035718854317],"[2.0, -0.8249339856566439]",0.091795,0.5,A1,2,0.183591
0,2,1.550589,0.197531,0.0,"[0.7368421052631575, 1.5789473684210522]",-8.591553,[11.059691242101714],"[2.0, -0.8234571171823121]",[11.059691242101714],"[2.0, -0.8234571171823121]",0.099562,0.5,A1,2,0.199124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,14,1.096953,inf,1.0,"[0.9473684210526314, 1.3684210526315788]",0.00007,[7.383090598305339e-05],"[1.0022532661802648, -0.9991764706299783]",[7.383090598305339e-05],"[1.0022532661802648, -0.9991764706299783]",5.324087,1.0,C2,61,324.769287
57,14,1.096953,inf,1.0,"[-0.9473684210526316, -1.5789473684210527]",0.00007,[7.235367814966039e-05],"[1.0022314515005382, -0.9991854282944734]",[7.235367814966039e-05],"[1.0022314515005382, -0.9991854282944734]",5.424478,1.0,C2,61,330.893132
58,14,1.096953,inf,1.0,"[0.10526315789473673, 1.1578947368421053]",0.000071,[7.259851785714584e-05],"[1.0022222828189662, -0.9991737200911809]",[7.235367814966039e-05],"[1.0022314515005382, -0.9991854282944734]",5.482382,1.0,C2,61,334.425277
